In [1]:
import argparse
import datetime
import glob
import hashlib
import json
import multiprocessing
import pickle
import os
import shutil
import subprocess
import uuid
import random

import numpy as np
import pandas as pd
import lightgbm as lgb
from collections import defaultdict
from lightgbm.sklearn import LGBMRanker
from tqdm import tqdm
import sys
sys.path.append('..')

from pyserini.analysis import Analyzer, get_lucene_analyzer
from pyserini.ltr import *
from pyserini.search import get_topics_with_reader

def train_data_loader(task='triple', neg_sample=10, cutoff=100, random_seed=12345):
    if task == 'triple' or task == 'rank':
        fn = f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle'
    elif task == 'rank_recall' or task == 'rank_hard':
        fn = f'train_{task}_sampled_with_{neg_sample}_{cutoff}_{random_seed}.pickle'
    else:
        raise Exception('unknown parameters')
    if os.path.exists(fn):
        sampled_train = pd.read_pickle(fn)
        print(sampled_train.shape)
        print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
        print(sampled_train.groupby('qid').count().mean())
        print(sampled_train.head(10))
        print(sampled_train.info())
        return sampled_train
    else:
        if task == 'triple':
            train = pd.read_csv('collections/msmarco-passage/qidpidtriples.train.full.2.tsv', sep="\t",
                                names=['qid', 'pos_pid', 'neg_pid'], dtype=np.int32)
            pos_half = train[['qid', 'pos_pid']].rename(columns={"pos_pid": "pid"}).drop_duplicates()
            pos_half['rel'] = np.int32(1)
            neg_half = train[['qid', 'neg_pid']].rename(columns={"neg_pid": "pid"}).drop_duplicates()
            neg_half['rel'] = np.int32(0)
            del train
            sampled_neg_half = []
            for qid, group in tqdm(neg_half.groupby('qid')):
                sampled_neg_half.append(group.sample(n=min(neg_sample, len(group)), random_state=random_seed))
            sampled_train = pd.concat([pos_half] + sampled_neg_half, axis=0, ignore_index=True)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        elif task == 'rank':
            qrel = defaultdict(list)
            with open("collections/msmarco-passage/qrels.train.tsv") as f:
                for line in f:
                    topicid, _, docid, rel = line.strip().split('\t')
                    assert rel == "1", line.split(' ')
                    qrel[topicid].append(docid)
            
            qid2pos = defaultdict(list)
            qid2neg = defaultdict(list)
            with open("runs/msmarco-passage/run.train.small.tsv") as f:
                for line in tqdm(f):
                    topicid, docid, rank = line.split()
                    assert topicid in qrel
                    if docid in qrel[topicid]:
                        qid2pos[topicid].append(docid)
                    else:
                        qid2neg[topicid].append(docid)
            sampled_train = []
            for topicid, pos_list in tqdm(qid2pos.items()):
                neg_list = random.sample(qid2neg[topicid], min(len(qid2neg[topicid]), neg_sample))
                for positive_docid in pos_list:
                    sampled_train.append((int(topicid), int(positive_docid), 1))
                for negative_docid in neg_list:
                    sampled_train.append((int(topicid), int(negative_docid), 0))
            sampled_train = pd.DataFrame(sampled_train,columns=['qid','pid','rel'],dtype=np.int32)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{random_seed}.pickle')
        elif task == 'rank_recall':
            qrel = defaultdict(list)
            with open("../collections/msmarco-passage/qrels.train.tsv") as f:
                for line in f:
                    topicid, _, docid, rel = line.strip().split('\t')
                    assert rel == "1", line.split(' ')
                    qrel[topicid].append(docid)
            
            qid2pos = defaultdict(list)
            qid2neg = defaultdict(list)
            with open("../runs/msmarco-passage/run.train.small.tsv") as f:
                for line in tqdm(f):
                    topicid, docid, rank = line.split()
                    assert topicid in qrel
                    if docid in qrel[topicid]:
                        qid2pos[topicid].append(docid)
                    else:
                        if int(rank) > cutoff:
                            qid2neg[topicid].append(docid)
            sampled_train = []
            for topicid, pos_list in tqdm(qid2pos.items()):
                neg_list = random.sample(qid2neg[topicid], min(len(qid2neg[topicid]), neg_sample))
                for positive_docid in pos_list:
                    sampled_train.append((int(topicid), int(positive_docid), 1))
                for negative_docid in neg_list:
                    sampled_train.append((int(topicid), int(negative_docid), 0))
            sampled_train = pd.DataFrame(sampled_train,columns=['qid','pid','rel'],dtype=np.int32)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{cutoff}_{random_seed}.pickle')
        elif task == 'rank_hard':
            qrel = defaultdict(list)
            with open("../collections/msmarco-passage/qrels.train.tsv") as f:
                for line in f:
                    topicid, _, docid, rel = line.strip().split('\t')
                    assert rel == "1", line.split(' ')
                    qrel[topicid].append(docid)
            
            qid2pos = defaultdict(list)
            qid2neg = defaultdict(list)
            with open("../runs/msmarco-passage/run.train.small.tsv") as f:
                for line in tqdm(f):
                    topicid, docid, rank = line.split()
                    assert topicid in qrel
                    if docid in qrel[topicid]:
                        qid2pos[topicid].append(docid)
                    else:
                        if int(rank) < cutoff:
                            qid2neg[topicid].append(docid)
            sampled_train = []
            for topicid, pos_list in tqdm(qid2pos.items()):
                neg_list = random.sample(qid2neg[topicid], min(len(qid2neg[topicid]), neg_sample))
                for positive_docid in pos_list:
                    sampled_train.append((int(topicid), int(positive_docid), 1))
                for negative_docid in neg_list:
                    sampled_train.append((int(topicid), int(negative_docid), 0))
            sampled_train = pd.DataFrame(sampled_train,columns=['qid','pid','rel'],dtype=np.int32)
            sampled_train = sampled_train.sort_values(['qid','pid']).set_index(['qid','pid'])
            print(sampled_train.shape)
            print(sampled_train.index.get_level_values('qid').drop_duplicates().shape)
            print(sampled_train.groupby('qid').count().mean())
            print(sampled_train.head(10))
            print(sampled_train.info())

            sampled_train.to_pickle(f'train_{task}_sampled_with_{neg_sample}_{cutoff}_{random_seed}.pickle')
        else:
            raise Exception('unknown parameters')
        return sampled_train
sampled_train = train_data_loader(task='triple',neg_sample=20)

(8385888, 1)
(400782,)
rel    20.923814
dtype: float64
             rel
qid pid         
3   689698     0
    970816     0
    1142680    1
    1519440    0
    1887112    0
    2006462    0
    2019206    0
    2605131    0
    2679073    0
    2963098    0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8385888 entries, (3, 689698) to (1185869, 8626607)
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   rel     int32
dtypes: int32(1)
memory usage: 303.9 MB
None


In [2]:
def dev_data_loader(task='pygaggle'):
    if os.path.exists(f'dev_{task}.pickle'):
        dev = pd.read_pickle(f'dev_{task}.pickle')
        print(dev.shape)
        print(dev.index.get_level_values('qid').drop_duplicates().shape)
        print(dev.groupby('qid').count().mean())
        print(dev.head(10))
        print(dev.info())
        dev_qrel = pd.read_pickle(f'dev_qrel.pickle')
        return dev, dev_qrel
    else:
        if task == 'rerank':
            dev = pd.read_csv('collections/msmarco-passage/top1000.dev', sep="\t",
                              names=['qid', 'pid', 'query', 'doc'], usecols=['qid', 'pid'], dtype=np.int32)
        elif task == 'anserini':
            dev = pd.read_csv('runs/msmarco-passage/run.msmarco-passage.dev.small.tsv',sep="\t",
                            names=['qid','pid','rank'], dtype=np.int32)
        elif task == 'pygaggle':
            dev = pd.read_csv('../pygaggle/data/msmarco_ans_entire/run.dev.small.tsv',sep="\t",
                            names=['qid','pid','rank'], dtype=np.int32)
        else:
            raise Exception('unknown parameters')
        dev_qrel = pd.read_csv('collections/msmarco-passage/qrels.dev.small.tsv', sep="\t",
                               names=["qid", "q0", "pid", "rel"], usecols=['qid', 'pid', 'rel'], dtype=np.int32)
        dev = dev.merge(dev_qrel, left_on=['qid', 'pid'], right_on=['qid', 'pid'], how='left')
        dev['rel'] = dev['rel'].fillna(0).astype(np.int32)
        dev = dev.sort_values(['qid','pid']).set_index(['qid','pid'])
        
        print(dev.shape)
        print(dev.index.get_level_values('qid').drop_duplicates().shape)
        print(dev.groupby('qid').count().mean())
        print(dev.head(10))
        print(dev.info())

        dev.to_pickle(f'dev_{task}.pickle')
        dev_qrel.to_pickle(f'dev_qrel.pickle')
        return dev, dev_qrel
dev, dev_qrel = dev_data_loader(task='pygaggle')

(6974598, 2)
(6980,)
rank    999.226074
rel     999.226074
dtype: float64
            rank  rel
qid pid              
2   55860    345    0
    72202    557    0
    72210    213    0
    98589    278    0
    98590    323    0
    98593    580    0
    98595    553    0
    112123   108    0
    112126   469    0
    112127    21    0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6974598 entries, (2, 55860) to (1102400, 8830447)
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   rank    int32
 1   rel     int32
dtypes: int32(2)
memory usage: 282.7 MB
None


In [3]:
def query_loader():
    queries = {}
    with open('queries.train.small.entity.json') as f:
        for line in f:
            query = json.loads(line)
            qid = query.pop('id')
            query['analyzed'] = query['analyzed'].split(" ")
            query['text'] = query['text_unlemm'].split(" ")
            query['text_unlemm'] = query['text_unlemm'].split(" ")
            query['text_bert_tok'] = query['text_bert_tok'].split(" ")
            assert 'raw' in query
            assert 'entity' in query
            queries[qid] = query
    with open('queries.dev.small.entity.json') as f:
        for line in f:
            query = json.loads(line)
            qid = query.pop('id')
            query['analyzed'] = query['analyzed'].split(" ")
            query['text'] = query['text_unlemm'].split(" ")
            query['text_unlemm'] = query['text_unlemm'].split(" ")
            query['text_bert_tok'] = query['text_bert_tok'].split(" ")
            assert 'raw' in query
            assert 'entity' in query
            queries[qid] = query
    with open('queries.eval.small.entity.json') as f:
        for line in f:
            query = json.loads(line)
            qid = query.pop('id')
            query['analyzed'] = query['analyzed'].split(" ")
            query['text'] = query['text_unlemm'].split(" ")
            query['text_unlemm'] = query['text_unlemm'].split(" ")
            query['text_bert_tok'] = query['text_bert_tok'].split(" ")
            assert 'raw' in query
            assert 'entity' in query
            queries[qid] = query
    return queries
queries = query_loader()

In [4]:
fe = FeatureExtractor('../../anserini/indexes/msmarco-passage/lucene-index-msmarco-flex_ent',max(multiprocessing.cpu_count()//2,1))

for qfield, ifield in [('analyzed','contents'),
                       ('text','text'),
                       ('text_unlemm','text_unlemm'),
                       ('text_bert_tok','text_bert_tok')]:
    print(qfield, ifield)
    fe.add(BM25(k1=0.9,b=0.4, field=ifield, qfield=qfield))
    fe.add(BM25(k1=1.2,b=0.75, field=ifield, qfield=qfield))
    fe.add(BM25(k1=2.0,b=0.75, field=ifield, qfield=qfield))

    fe.add(LMDir(mu=1000, field=ifield, qfield=qfield))
    fe.add(LMDir(mu=1500, field=ifield, qfield=qfield))
    fe.add(LMDir(mu=2500, field=ifield, qfield=qfield))

    fe.add(LMJM(0.1, field=ifield, qfield=qfield))
    fe.add(LMJM(0.4, field=ifield, qfield=qfield))
    fe.add(LMJM(0.7, field=ifield, qfield=qfield))

    fe.add(NTFIDF(field=ifield, qfield=qfield))
    fe.add(ProbalitySum(field=ifield, qfield=qfield))

    fe.add(DFR_GL2(field=ifield, qfield=qfield))
    fe.add(DFR_In_expB2(field=ifield, qfield=qfield))
    fe.add(DPH(field=ifield, qfield=qfield))

    fe.add(Proximity(field=ifield, qfield=qfield))
    fe.add(TPscore(field=ifield, qfield=qfield))
    fe.add(tpDist(field=ifield, qfield=qfield))

    fe.add(DocSize(field=ifield))

    fe.add(QueryLength(qfield=qfield))
    fe.add(QueryCoverageRatio(qfield=qfield))
    fe.add(UniqueTermCount(qfield=qfield))
    fe.add(MatchingTermCount(field=ifield, qfield=qfield))
    fe.add(SCS(field=ifield, qfield=qfield))

    fe.add(tfStat(AvgPooler(), field=ifield, qfield=qfield))
    fe.add(tfStat(MedianPooler(), field=ifield, qfield=qfield))
    fe.add(tfStat(SumPooler(), field=ifield, qfield=qfield))
    fe.add(tfStat(MinPooler(), field=ifield, qfield=qfield))
    fe.add(tfStat(MaxPooler(), field=ifield, qfield=qfield))
    fe.add(tfStat(VarPooler(), field=ifield, qfield=qfield))
    fe.add(tfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))
    fe.add(tfStat(ConfidencePooler(), field=ifield, qfield=qfield))

    fe.add(tfIdfStat(AvgPooler(), field=ifield, qfield=qfield))
    fe.add(tfIdfStat(MedianPooler(), field=ifield, qfield=qfield))
    fe.add(tfIdfStat(SumPooler(), field=ifield, qfield=qfield))
    fe.add(tfIdfStat(MinPooler(), field=ifield, qfield=qfield))
    fe.add(tfIdfStat(MaxPooler(), field=ifield, qfield=qfield))
    fe.add(tfIdfStat(VarPooler(), field=ifield, qfield=qfield))
    fe.add(tfIdfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))
    fe.add(tfIdfStat(ConfidencePooler(), field=ifield, qfield=qfield))

    fe.add(scqStat(AvgPooler(), field=ifield, qfield=qfield))
    fe.add(scqStat(MedianPooler(), field=ifield, qfield=qfield))
    fe.add(scqStat(SumPooler(), field=ifield, qfield=qfield))
    fe.add(scqStat(MinPooler(), field=ifield, qfield=qfield))
    fe.add(scqStat(MaxPooler(), field=ifield, qfield=qfield))
    fe.add(scqStat(VarPooler(), field=ifield, qfield=qfield))
    fe.add(scqStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))
    fe.add(scqStat(ConfidencePooler(), field=ifield, qfield=qfield))

    fe.add(normalizedTfStat(AvgPooler(), field=ifield, qfield=qfield))
    fe.add(normalizedTfStat(MedianPooler(), field=ifield, qfield=qfield))
    fe.add(normalizedTfStat(SumPooler(), field=ifield, qfield=qfield))
    fe.add(normalizedTfStat(MinPooler(), field=ifield, qfield=qfield))
    fe.add(normalizedTfStat(MaxPooler(), field=ifield, qfield=qfield))
    fe.add(normalizedTfStat(VarPooler(), field=ifield, qfield=qfield))
    fe.add(normalizedTfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))
    fe.add(normalizedTfStat(ConfidencePooler(), field=ifield, qfield=qfield))

    fe.add(idfStat(AvgPooler(), field=ifield, qfield=qfield))
    fe.add(idfStat(MedianPooler(), field=ifield, qfield=qfield))
    fe.add(idfStat(SumPooler(), field=ifield, qfield=qfield))
    fe.add(idfStat(MinPooler(), field=ifield, qfield=qfield))
    fe.add(idfStat(MaxPooler(), field=ifield, qfield=qfield))
    fe.add(idfStat(VarPooler(), field=ifield, qfield=qfield))
    fe.add(idfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))
    fe.add(idfStat(ConfidencePooler(), field=ifield, qfield=qfield))

    fe.add(ictfStat(AvgPooler(), field=ifield, qfield=qfield))
    fe.add(ictfStat(MedianPooler(), field=ifield, qfield=qfield))
    fe.add(ictfStat(SumPooler(), field=ifield, qfield=qfield))
    fe.add(ictfStat(MinPooler(), field=ifield, qfield=qfield))
    fe.add(ictfStat(MaxPooler(), field=ifield, qfield=qfield))
    fe.add(ictfStat(VarPooler(), field=ifield, qfield=qfield))
    fe.add(ictfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))
    fe.add(ictfStat(ConfidencePooler(), field=ifield, qfield=qfield))

    fe.add(UnorderedSequentialPairs(3, field=ifield, qfield=qfield))
    fe.add(UnorderedSequentialPairs(8, field=ifield, qfield=qfield))
    fe.add(UnorderedSequentialPairs(15, field=ifield, qfield=qfield))
    fe.add(OrderedSequentialPairs(3, field=ifield, qfield=qfield))
    fe.add(OrderedSequentialPairs(8, field=ifield, qfield=qfield))
    fe.add(OrderedSequentialPairs(15, field=ifield, qfield=qfield))
    fe.add(UnorderedQueryPairs(3, field=ifield, qfield=qfield))
    fe.add(UnorderedQueryPairs(8, field=ifield, qfield=qfield))
    fe.add(UnorderedQueryPairs(15, field=ifield, qfield=qfield))
    fe.add(OrderedQueryPairs(3, field=ifield, qfield=qfield))
    fe.add(OrderedQueryPairs(8, field=ifield, qfield=qfield))
    fe.add(OrderedQueryPairs(15, field=ifield, qfield=qfield))

# fe.add(EntityHowLong())
# fe.add(EntityHowMany())
# fe.add(EntityHowMuch())
# fe.add(EntityWhen())
# fe.add(EntityWhere())
# fe.add(EntityWho())
# fe.add(EntityWhereMatch())
# fe.add(EntityWhoMatch())

# for ent_type in ['PERSON','NORP','FAC','ORG','GPE','LOC','PRODUCT','EVENT','WORK_OF_ART','LAW',
#                 'LANGUAGE','DATE','TIME','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']:
#     fe.add(EntityDocCount(ent_type))

# fe.add(QueryRegex("^[0-9.+_ ]*what.*$"))

fe.add(IBMModel1("../FlexNeuART/collections/msmarco_doc/derived_data/giza/title_unlemm","text_unlemm","title_unlemm","text_unlemm"))
print('IBM model Loaded')
fe.add(IBMModel1("../FlexNeuART/collections/msmarco_doc/derived_data/giza/url_unlemm","text_unlemm","url_unlemm","text_unlemm"))
print('IBM model Loaded')
fe.add(IBMModel1("../FlexNeuART/collections/msmarco_doc/derived_data/giza/body","text_unlemm","body","text_unlemm"))
print('IBM model Loaded')
fe.add(IBMModel1("../FlexNeuART/collections/msmarco_doc/derived_data/giza/text_bert_tok","text_bert_tok","text_bert_tok","text_bert_tok"))
print('IBM model Loaded')

analyzed contents
text text
text_unlemm text_unlemm
text_bert_tok text_bert_tok
IBM model Loaded
IBM model Loaded
IBM model Loaded
IBM model Loaded


In [5]:
def batch_extract(df, queries, fe):
    tasks = []
    task_infos = []
    group_lst = []

    info_dfs = []
    feature_dfs = []
    group_dfs = []

    for qid, group in tqdm(df.groupby('qid')):
        task = {
            "qid": str(qid),
            "docIds": [],
            "rels": [],
            "query_dict": queries[str(qid)]
        }
        for t in group.reset_index().itertuples():
            task["docIds"].append(str(t.pid))
            task_infos.append((qid, t.pid, t.rel))
        tasks.append(task)
        group_lst.append((qid, len(task['docIds'])))
        if len(tasks) == 10000:
            features = fe.batch_extract(tasks)
            task_infos = pd.DataFrame(task_infos, columns=['qid', 'pid', 'rel'])
            group = pd.DataFrame(group_lst, columns=['qid', 'count'])
            print(features.shape)
            print(task_infos.qid.drop_duplicates().shape)
            print(group.mean())
            print(features.head(10))
            print(features.info())
            info_dfs.append(task_infos)
            feature_dfs.append(features)
            group_dfs.append(group)
            tasks = []
            task_infos = []
            group_lst = []
    # deal with rest
    if len(tasks) > 0:
        features = fe.batch_extract(tasks)
        task_infos = pd.DataFrame(task_infos, columns=['qid', 'pid', 'rel'])
        group = pd.DataFrame(group_lst, columns=['qid', 'count'])
        print(features.shape)
        print(task_infos.qid.drop_duplicates().shape)
        print(group.mean())
        print(features.head(10))
        print(features.info())
        info_dfs.append(task_infos)
        feature_dfs.append(features)
        group_dfs.append(group)
    info_dfs = pd.concat(info_dfs, axis=0, ignore_index=True)
    feature_dfs = pd.concat(feature_dfs, axis=0, ignore_index=True, copy=False)
    group_dfs = pd.concat(group_dfs, axis=0, ignore_index=True)
    return info_dfs, feature_dfs, group_dfs

In [6]:
def hash_df(df):
    h = pd.util.hash_pandas_object(df)
    return hex(h.sum().astype(np.uint64))


def hash_anserini_jar():
    find = glob.glob(os.environ['ANSERINI_CLASSPATH'] + "/*fatjar.jar")
    assert len(find) == 1
    md5Hash = hashlib.md5(open(find[0], 'rb').read())
    return md5Hash.hexdigest()


def hash_fe(fe):
    return hashlib.md5(','.join(sorted(fe.feature_names())).encode()).hexdigest()


def data_loader(task, df, queries, fe):
    df_hash = hash_df(df)
    jar_hash = hash_anserini_jar()
    fe_hash = hash_fe(fe)
    if os.path.exists(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle'):
        res = pickle.load(open(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle', 'rb'))
        print(res['info'].shape)
        print(res['info'].qid.drop_duplicates().shape)
        print(res['group'].mean())
        return res
    else:
        if task == 'train' or task == 'dev':
            info, data, group = batch_extract(df, queries, fe)
            obj = {'info':info, 'data': data, 'group': group,
                   'df_hash': df_hash, 'jar_hash': jar_hash, 'fe_hash': fe_hash}
            print(info.shape)
            print(info.qid.drop_duplicates().shape)
            print(group.mean())
            pickle.dump(obj, open(f'{task}_{df_hash}_{jar_hash}_{fe_hash}.pickle', 'wb'))
            return obj
        else:
            raise Exception('unknown parameters')

In [7]:
import json
def export(df, fn, queries):
    with open(fn,'w') as f:
        line_num = 0
        for qid, group in tqdm(df.groupby('qid')):
            line = {}
            line['qid'] = qid
            line['docIds'] = [str(did) for did in group.reset_index().pid.drop_duplicates().tolist()]
            assert 'qid' not in queries[str(qid)]
            assert 'docIds' not in queries[str(qid)]
            line.update(queries[str(qid)])
            f.write(json.dumps(line)+'\n')
            line_num += 1

In [8]:
train_extracted = data_loader('train', sampled_train, queries, fe)
dev_extracted = data_loader('dev', dev, queries, fe)
del sampled_train, dev
feature_name = fe.feature_names()
del queries, fe

  3%|▎         | 10068/400782 [04:02<239:31:07,  2.21s/it]

(210283, 336)
(10000,)
qid      16070.1173
count       21.0283
dtype: float64
   contents_analyzed_BM25_k1_0.90_b_0.40  \
0                              16.065151   
1                              22.893564   
2                              26.467836   
3                              16.092194   
4                              17.071817   
5                              26.280588   
6                              19.257418   
7                              13.279017   
8                              13.294892   
9                              19.064749   

   contents_analyzed_BM25_k1_1.20_b_0.75  \
0                              16.743813   
1                              23.464577   
2                              27.822981   
3                              17.323364   
4                              18.976582   
5                              28.163824   
6                              21.266153   
7                              15.103254   
8                              13.668623 

  5%|▌         | 20081/400782 [04:38<27:27:55,  3.85it/s] 

(210507, 336)
(10000,)
qid      43586.6803
count       21.0507
dtype: float64
   contents_analyzed_BM25_k1_0.90_b_0.40  \
0                              12.922753   
1                              12.513888   
2                              12.434628   
3                              13.131429   
4                              14.483192   
5                              12.844487   
6                              20.265697   
7                              18.062141   
8                              17.035084   
9                              13.037727   

   contents_analyzed_BM25_k1_1.20_b_0.75  \
0                              14.139851   
1                              13.831271   
2                              13.694565   
3                              14.182406   
4                              15.425590   
5                              13.905419   
6                              21.310802   
7                              19.409094   
8                              17.027304 

  8%|▊         | 30206/400782 [05:04<2:32:47, 40.42it/s] 

...

(16461, 336)
(782,)
qid      1.185035e+06
count    2.104987e+01
dtype: float64
   contents_analyzed_BM25_k1_0.90_b_0.40  \
0                              13.269936   
1                              10.223487   
2                              11.566961   
3                              11.202425   
4                               9.792343   
5                              10.503739   
6                              11.671648   
7                              16.880190   
8                              10.664545   
9                              15.841070   

   contents_analyzed_BM25_k1_1.20_b_0.75  \
0                              13.485291   
1                              11.868622   
2                              12.111023   
3                              12.574932   
4                              10.688994   
5                              11.334428   
6                              11.046934   
7                              15.679298   
8                              11.747111

(8385888, 3)
(400782,)
qid      594038.702983
count        20.923814
dtype: float64


100%|██████████| 6980/6980 [00:18<00:00, 386.90it/s]


(6974598, 336)
(6980,)
qid      741638.817908
count       999.226074
dtype: float64
   contents_analyzed_BM25_k1_0.90_b_0.40  \
0                              12.348820   
1                              10.927653   
2                              13.675473   
3                              12.699286   
4                              12.492470   
5                              11.077914   
6                              11.181725   
7                              15.955744   
8                              11.468307   
9                              21.200821   

   contents_analyzed_BM25_k1_1.20_b_0.75  \
0                              13.151269   
1                              11.041017   
2                              13.716912   
3                              14.042357   
4                              13.508439   
5                              11.527296   
6                              11.772854   
7                              17.375216   
8                              12.4

(6974598, 3)
(6980,)
qid      741638.817908
count       999.226074
dtype: float64


In [9]:
def eval_mrr(dev_data):
    score_tie_counter = 0
    score_tie_query = set()

    MRR = []
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM

        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            prev_pid = t.pid
            rank += 1
            if t.rel>0:
                MRR.append(1.0/rank)
                break
            elif rank == 10 or rank == len(group):
                MRR.append(0.)
                break

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie,np.mean(MRR))


In [10]:
def eval_recall(dev_qrel, dev_data):
    dev_rel_num = dev_qrel[dev_qrel['rel']>0].groupby('qid').count()['rel']

    score_tie_counter = 0
    score_tie_query = set()
    
    recall_point = [10,20,50,100,200,500,1000]
    recall_curve = {k:[] for k in recall_point}
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM
        total_rel = dev_rel_num.loc[qid]
        query_recall = [0 for k in recall_point]
        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            rank += 1
            if t.rel>0:
                for i,p in enumerate(recall_point):
                    if rank <= p:
                        query_recall[i] += 1
        for i,p in enumerate(recall_point):
            if total_rel>0:
                recall_curve[p].append(query_recall[i]/total_rel)
            else:
                recall_curve[p].append(0.)

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie)
    
    for k,v in recall_curve.items():
        avg = np.mean(v)
        print(f'recall@{k}:{avg}')


In [11]:
for i,n in enumerate(feature_name):
    if np.isnan(train_extracted['data'].iloc[:,i]).any():
        print(n)
        print(train_extracted['info'].loc[train_extracted['data'].iloc[:,i].isna(),
                                          ['qid','pid']].head(10))

text_text_LMJM_lambda_0.10
            qid      pid
17745      3671  6416650
164965    24130  4209198
532270    72865  5018776
551592    75806  5323609
578451    79357  8026198
693707    95916  4705000
1081291  149467  2267954
1307485  180247  2872095
1320859  181991  7165701
1357277  187376  7229004
text_text_LMJM_lambda_0.40
            qid      pid
17745      3671  6416650
164965    24130  4209198
532270    72865  5018776
551592    75806  5323609
578451    79357  8026198
693707    95916  4705000
1081291  149467  2267954
1307485  180247  2872095
1320859  181991  7165701
1357277  187376  7229004
text_text_LMJM_lambda_0.70
            qid      pid
17745      3671  6416650
164965    24130  4209198
532270    72865  5018776
551592    75806  5323609
578451    79357  8026198
693707    95916  4705000
1081291  149467  2267954
1307485  180247  2872095
1320859  181991  7165701
1357277  187376  7229004
text_text_Prob
            qid      pid
17745      3671  6416650
164965    24130  4209198
5322

In [12]:
def gen_dev_group_rel_num(dev_qrel, dev_extracted):
    dev_rel_num = dev_qrel[dev_qrel['rel']>0].groupby('qid').count()['rel']
    prev_qid = None
    dev_rel_num_list = []
    for t in dev_extracted['info'].itertuples():
        if prev_qid is None or t.qid != prev_qid:
            prev_qid = t.qid
            dev_rel_num_list.append(dev_rel_num.loc[t.qid])
        else:
            continue
    assert len(dev_rel_num_list) == dev_qrel.qid.drop_duplicates().shape[0]

    def recall_at_200(preds, dataset):
        labels = dataset.get_label()
        groups = dataset.get_group()
        idx = 0
        recall = 0
        assert len(dev_rel_num_list) == len(groups)
        for g, gnum in zip(groups, dev_rel_num_list):
            top_preds = labels[idx:idx + g][np.argsort(preds[idx:idx + g])]
            recall += np.sum(top_preds[-200:]) / gnum
            idx += g
        assert idx == len(preds)
        return 'recall@200', recall / len(groups), True

    return recall_at_200
eval_fn = gen_dev_group_rel_num(dev_qrel, dev_extracted)

In [13]:
lgb_train = lgb.Dataset(train_extracted['data'].loc[:, feature_name],
                            label=train_extracted['info']['rel'],
                            group=train_extracted['group']['count'])
lgb_valid = lgb.Dataset(dev_extracted['data'].loc[:, feature_name],
                        label=dev_extracted['info']['rel'],
                        group=dev_extracted['group']['count'],
                        free_raw_data=False)

In [14]:
import re
filtered_feature_name = list(map(lambda x:re.sub('[^A-Za-z0-9_]+', '', x), feature_name))

In [18]:
dev_extracted['info']['score'] = 0.
for seed in [12345]:
    params = {
            'boosting_type': 'goss',
            'objective': 'lambdarank',
            'max_bin':255,
            'num_leaves':100,
            'max_depth':-1,
            'min_data_in_leaf':50,
            'min_sum_hessian_in_leaf':0,
#             'bagging_fraction':0.8,
#             'bagging_freq':50,
#             'feature_fraction':0.5,
            'learning_rate':0.1,
            'num_boost_round':1000,
            'early_stopping_round':200,
            'metric':'custom',
            'label_gain':[0,1],
            'lambdarank_truncation_level':20,
            'lambdarank_norm':True,
            'seed':seed,
            'num_threads':max(multiprocessing.cpu_count()//2,1)
    }
    num_boost_round = params.pop('num_boost_round')
    early_stopping_round = params.pop('early_stopping_round')
    gbm = lgb.train(params, lgb_train,
                    valid_sets=lgb_valid,
                    num_boost_round=num_boost_round,
                    early_stopping_rounds=early_stopping_round,
                    feval=eval_fn,
                    feature_name=filtered_feature_name,
                    verbose_eval=True)

    dev_extracted['info']['score'] = gbm.predict(lgb_valid.get_data())
    best_score = gbm.best_score['valid_0']['recall@200']
    print(best_score)
    best_iteration = gbm.best_iteration
    print(best_iteration)
    feature_importances = sorted(list(zip(feature_name,gbm.feature_importance().tolist())),
                                 key=lambda x:x[1],reverse=True)
    print(feature_importances)
eval_recall(dev_qrel, dev_extracted['info'])
eval_mrr(dev_extracted['info'])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 11.930070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 68509
[LightGBM] [Info] Number of data points in the train set: 8385888, number of used features: 336
[LightGBM] [Info] Using GOSS
[1]	valid_0's recall@200: 0.737954
Training until validation scores don't improve for 200 rounds
[2]	valid_0's recall@200: 0.741965
[3]	valid_0's recall@200: 0.746036
[4]	valid_0's recall@200: 0.746335
[5]	valid_0's recall@200: 0.750215
[6]	valid_0's recall@200: 0.754465
[7]	valid_0's recall@200: 0.757426
[8]	valid_0's recall@200: 0.758309
[9]	valid_0's recall@200: 0.762369
[10]	valid_0's recall@200: 0.764959
[11]	valid_0's recall@200: 0.768995
[12]	valid_0's recall@200: 0.770738
[13]	valid_0's recall@200: 0.772385
[14]	valid_0's recall@200: 0.772194
[15]	valid_0's recall@200: 0.773914
[16]	valid_0's recall@200: 0.774558
[17]	valid_0's recall@200: 0.775609
[18]	vali

[216]	valid_0's recall@200: 0.803044
[217]	valid_0's recall@200: 0.803188
[218]	valid_0's recall@200: 0.803116
[219]	valid_0's recall@200: 0.802615
[220]	valid_0's recall@200: 0.802615
[221]	valid_0's recall@200: 0.802615
[222]	valid_0's recall@200: 0.802471
[223]	valid_0's recall@200: 0.802328
[224]	valid_0's recall@200: 0.802543
[225]	valid_0's recall@200: 0.802543
[226]	valid_0's recall@200: 0.80283
[227]	valid_0's recall@200: 0.802901
[228]	valid_0's recall@200: 0.803044
[229]	valid_0's recall@200: 0.803044
[230]	valid_0's recall@200: 0.803044
[231]	valid_0's recall@200: 0.802901
[232]	valid_0's recall@200: 0.802901
[233]	valid_0's recall@200: 0.802901
[234]	valid_0's recall@200: 0.802901
[235]	valid_0's recall@200: 0.80283
[236]	valid_0's recall@200: 0.802973
[237]	valid_0's recall@200: 0.80283
[238]	valid_0's recall@200: 0.802973
[239]	valid_0's recall@200: 0.802973
[240]	valid_0's recall@200: 0.803116
[241]	valid_0's recall@200: 0.803259
[242]	valid_0's recall@200: 0.803331
[243

[438]	valid_0's recall@200: 0.80585
[439]	valid_0's recall@200: 0.80585
[440]	valid_0's recall@200: 0.80585
[441]	valid_0's recall@200: 0.80585
[442]	valid_0's recall@200: 0.806137
[443]	valid_0's recall@200: 0.806137
[444]	valid_0's recall@200: 0.806137
[445]	valid_0's recall@200: 0.805993
[446]	valid_0's recall@200: 0.80628
[447]	valid_0's recall@200: 0.80628
[448]	valid_0's recall@200: 0.806423
[449]	valid_0's recall@200: 0.80628
[450]	valid_0's recall@200: 0.806137
[451]	valid_0's recall@200: 0.80628
[452]	valid_0's recall@200: 0.80628
[453]	valid_0's recall@200: 0.806137
[454]	valid_0's recall@200: 0.805993
[455]	valid_0's recall@200: 0.806137
[456]	valid_0's recall@200: 0.806065
[457]	valid_0's recall@200: 0.806065
[458]	valid_0's recall@200: 0.806065
[459]	valid_0's recall@200: 0.805778
[460]	valid_0's recall@200: 0.805778
[461]	valid_0's recall@200: 0.805922
[462]	valid_0's recall@200: 0.806065
[463]	valid_0's recall@200: 0.805922
[464]	valid_0's recall@200: 0.806065
[465]	vali

[661]	valid_0's recall@200: 0.805635
[662]	valid_0's recall@200: 0.805635
[663]	valid_0's recall@200: 0.805635
[664]	valid_0's recall@200: 0.805778
[665]	valid_0's recall@200: 0.805778
[666]	valid_0's recall@200: 0.805778
[667]	valid_0's recall@200: 0.805778
[668]	valid_0's recall@200: 0.80585
[669]	valid_0's recall@200: 0.80585
[670]	valid_0's recall@200: 0.80585
[671]	valid_0's recall@200: 0.80585
[672]	valid_0's recall@200: 0.80585
[673]	valid_0's recall@200: 0.80585
[674]	valid_0's recall@200: 0.805707
[675]	valid_0's recall@200: 0.80585
[676]	valid_0's recall@200: 0.80585
[677]	valid_0's recall@200: 0.80585
[678]	valid_0's recall@200: 0.80585
[679]	valid_0's recall@200: 0.805993
[680]	valid_0's recall@200: 0.805993
[681]	valid_0's recall@200: 0.805993
[682]	valid_0's recall@200: 0.806137
[683]	valid_0's recall@200: 0.80585
[684]	valid_0's recall@200: 0.80585
[685]	valid_0's recall@200: 0.805993
[686]	valid_0's recall@200: 0.805993
[687]	valid_0's recall@200: 0.806137
[688]	valid_0

100%|██████████| 6980/6980 [00:18<00:00, 377.22it/s]


score_tie occurs 180865 times in 6780 queries
recall@10:0.48933858643744027
recall@20:0.5828319006685769
recall@50:0.6869866284622732
recall@100:0.7529727793696275
recall@200:0.8067812798471825
recall@500:0.8464302769818529
recall@1000:0.8573424068767909


100%|██████████| 6980/6980 [00:10<00:00, 647.51it/s]

score_tie occurs 210 times in 176 queries 0.2501120548505935
